In [ ]:
pip install numpy pandas nltk

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
credits_df = pd.read_csv("credits.csv")
movies_df = pd.read_csv("movies.csv")

In [ ]:
credits_df.shape

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
movies_df = movies_df.merge(credits_df, on = 'title')
movies_df.shape

In [ ]:
movies_df.head()

In [ ]:
movies_df = movies_df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies_df.head()

In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.dropna(inplace=True)
movies_df.drop_duplicates(inplace=True)

In [ ]:
movies_df.duplicated().sum()

In [ ]:
movies_df.iloc[0].genres

In [ ]:
import ast
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(convert)
movies_df['keywords'] = movies_df['keywords'].apply(convert)
movies_df.head()


In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [ ]:
movies_df['cast'] = movies_df['cast'].apply(convert3)
movies_df.head()

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies_df['crew'] = movies_df['crew'].apply(fetch_director)
movies_df.head()

In [ ]:
movies_df['overview'] = movies_df['overview'].apply(lambda x:x.split())
movies_df.head()

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda x:[i.replace(' ', '') for i in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x:[i.replace(' ', '') for i in x])
movies_df['cast'] = movies_df['cast'].apply(lambda x:[i.replace(' ', '') for i in x])
movies_df['crew'] = movies_df['crew'].apply(lambda x:[i.replace(' ', '') for i in x])

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [ ]:
movies_df.head()

In [23]:
movies_df['tags'] = movies_df['overview'] + movies_df['genres'] + movies_df['cast'] + movies_df['keywords'] + movies_df['crew']

In [ ]:
movies_df.head()

In [ ]:
new_df = movies_df[['movie_id', 'title', 'tags']]
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(5000,))

In [33]:

import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [34]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vectors)

In [37]:
similarity = cosine_similarity(vectors)
similarity[0]

array([1.        , 0.08980265, 0.05986843, ..., 0.02457366, 0.02635231,
       0.        ], shape=(4805,))

In [40]:
sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x:x[1])[1:6]

[(539, np.float64(0.25724787771376323)),
 (1192, np.float64(0.2545875386086578)),
 (260, np.float64(0.24759378423606915)),
 (507, np.float64(0.24720661623652207)),
 (1214, np.float64(0.24595492912420727))]

In [41]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse= True, key = lambda x:x[1])[1:10]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [44]:
recommend('Iron Man')

Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers
Ant-Man
X-Men
X-Men: The Last Stand
Thor: The Dark World
